In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as engine
#load_dotenv()

# Dataset: Diversity of United States Counties (GENERAL DATA)

Content: Diversity Index of Every US County using the Simpson Diversity Index: D = 1 - ∑(n/N)^2 (where n = number of people of a given race and N is the total number of people of all races, to get the probability of randomly selecting two people and getting two people of different races (ecological entropy))

# Dataset:  Covid 19 Infection / Death Data Set (US COUNTIES)
The file contains the cumulative count of confirmed, death and recovered cases of COVID-19 from different countries from 22nd January 2020 to April 9 2020.

# FOCUS: FLORIDA COUNTIES

### Extract CSVs into DataFrames

In [10]:
county_death = pd.read_csv("Resources/usa_county_wise.csv")
df1 = county_death.copy()
df1.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/20,0,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/20,0,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/20,0,0


### Extract XLSX into DataFrames

In [7]:
#reading new dataset
diversity = pd.read_excel("Resources/diversityindex.xlsx")
df2 = diversity.copy()
df2.head()

,Location,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9


# Transform Dataset 1

In [11]:
#drop irrevelant columns
df1 = df1.drop(['Lat', 'Long_', 'UID', 'iso2', 'iso3','code3','FIPS','Admin2'], axis=1)

In [12]:
df1

,Province_State,Country_Region,Combined_Key,Date,Confirmed,Deaths
0,American Samoa,US,"American Samoa, US",1/22/20,0,0
1,Guam,US,"Guam, US",1/22/20,0,0
2,Northern Mariana Islands,US,"Northern Mariana Islands, US",1/22/20,0,0
3,Puerto Rico,US,"Puerto Rico, US",1/22/20,0,0
4,Virgin Islands,US,"Virgin Islands, US",1/22/20,0,0
...,...,...,...,...,...,...
293485,Utah,US,"Central Utah, Utah, US",4/20/20,17,0
293486,Utah,US,"Southeast Utah, Utah, US",4/20/20,7,0
293487,Utah,US,"Southwest Utah, Utah, US",4/20/20,70,1
293488,Utah,US,"TriCounty, Utah, US",4/20/20,10,0


In [14]:
#split Combined_Key column to seperate county name into it's own column
split = df1["Combined_Key"].str.split(",", n = 2, expand = True)


In [15]:
#reviewing the split process
split

,0,1,2
0,American Samoa,US,None
1,Guam,US,None
2,Northern Mariana Islands,US,None
3,Puerto Rico,US,None
4,Virgin Islands,US,None
...,...,...,...
293485,Central Utah,Utah,US
293486,Southeast Utah,Utah,US
293487,Southwest Utah,Utah,US
293488,TriCounty,Utah,US


In [16]:
#adding the new column to the dataframe
df1['County'] = split[0]

In [17]:
df1

,Province_State,Country_Region,Combined_Key,Date,Confirmed,Deaths,County
0,American Samoa,US,"American Samoa, US",1/22/20,0,0,American Samoa
1,Guam,US,"Guam, US",1/22/20,0,0,Guam
2,Northern Mariana Islands,US,"Northern Mariana Islands, US",1/22/20,0,0,Northern Mariana Islands
3,Puerto Rico,US,"Puerto Rico, US",1/22/20,0,0,Puerto Rico
4,Virgin Islands,US,"Virgin Islands, US",1/22/20,0,0,Virgin Islands
...,...,...,...,...,...,...,...
293485,Utah,US,"Central Utah, Utah, US",4/20/20,17,0,Central Utah
293486,Utah,US,"Southeast Utah, Utah, US",4/20/20,7,0,Southeast Utah
293487,Utah,US,"Southwest Utah, Utah, US",4/20/20,70,1,Southwest Utah
293488,Utah,US,"TriCounty, Utah, US",4/20/20,10,0,TriCounty


In [19]:
#drop Combined_Key column
df1 = df1.drop(['Combined_Key'], axis=1)

In [21]:
#focus on Florida
df1_florida = df1[df1['Province_State'] == 'Florida']
df1_florida

,Province_State,Country_Region,Date,Confirmed,Deaths,County
325,Florida,US,1/22/20,0,0,Alachua
326,Florida,US,1/22/20,0,0,Baker
327,Florida,US,1/22/20,0,0,Bay
328,Florida,US,1/22/20,0,0,Bradford
329,Florida,US,1/22/20,0,0,Brevard
...,...,...,...,...,...,...
290618,Florida,US,4/20/20,15,1,Wakulla
290619,Florida,US,4/20/20,32,0,Walton
290620,Florida,US,4/20/20,6,0,Washington
293387,Florida,US,4/20/20,0,0,Out of FL


In [22]:
#group by county 
df_bycounty = df1_florida.groupby(['County']).agg('max')

In [23]:
df_bycounty

,Province_State,Country_Region,Date,Confirmed,Deaths
County,,,,,
Alachua,Florida,US,4/9/20,218,0
Baker,Florida,US,4/9/20,17,3
Bay,Florida,US,4/9/20,57,2
Bradford,Florida,US,4/9/20,42,2
Brevard,Florida,US,4/9/20,218,7
...,...,...,...,...,...
Union,Florida,US,4/9/20,3,0
Volusia,Florida,US,4/9/20,342,10
Wakulla,Florida,US,4/9/20,15,2


In [24]:
df1 = df_bycounty.copy()

# Transform Dataset 2

In [113]:
#second dataset, general diversity stats 2013

In [25]:
df2.head()

,Location,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9


In [26]:
#column rename
df2 = df2.rename(columns=\
                            {'Black or African American alone, percent, 2013':'Black',\
                             'American Indian and Alaska Native alone, percent, 2013': 'Native American',\
                             'Asian alone, percent, 2013' : 'Asian',\
                             'Native Hawaiian and Other Pacific Islander alone, percent,' : 'Pacific Islander',\
                             'Two or More Races, percent, 2013' : 'Plus two races',\
                             'Hispanic or Latino, percent, 2013' : 'Hispanic',\
                             'White alone, not Hispanic or Latino, percent, 2013' : 'White'})

In [37]:
#split location into county and state

In [27]:
split2 = df2["Location"].str.split(",", n = 1, expand = True)


In [28]:
split2

,0,1
0,Aleutians West Census Area,AK
1,Queens County,NY
2,Maui County,HI
3,Alameda County,CA
4,Aleutians East Borough,AK
...,...,...
3190,Osage County,MO
3191,Lincoln County,WV
3192,Leslie County,KY
3193,Blaine County,NE


In [45]:
#adding the new columns

In [29]:
df2['County']=split2[0]

In [30]:
df2['State']=split2[1]

In [31]:
df2

,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,County,State
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,Aleutians West Census Area,AK
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,Queens County,NY
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,Maui County,HI
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,Alameda County,CA
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,Aleutians East Borough,AK
...,...,...,...,...,...,...,...,...,...,...,...
3190,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1,Osage County,MO
3191,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2,Lincoln County,WV
3192,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2,Leslie County,KY
3193,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8,Blaine County,NE


In [32]:
# FOCUS on florida and remove  leading spacing in column name
diversity_FL = df2[df2['State'] == ' FL'].copy()

In [33]:
diversity_FL['State'] = diversity_FL['State'].str.lstrip()

In [34]:
df2 = diversity_FL.copy()

In [35]:
df2

,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,County,State
31,"Broward County, FL",0.678237,28.5,0.4,3.6,0.1,2.0,26.9,40.8,Broward County,FL
34,"Orange County, FL",0.671218,22.0,0.6,5.4,0.2,2.4,28.7,44.1,Orange County,FL
71,"Hillsborough County, FL",0.626423,17.4,0.5,3.9,0.1,2.5,26.0,52.3,Hillsborough County,FL
96,"Hendry County, FL",0.609280,13.4,2.2,1.1,0.2,1.1,50.3,34.5,Hendry County,FL
103,"Osceola County, FL",0.605921,13.1,0.8,3.0,0.2,2.5,48.6,37.3,Osceola County,FL
...,...,...,...,...,...,...,...,...,...,...,...
1663,"Dixie County, FL",0.257861,8.9,0.4,0.4,0.0,1.5,3.5,85.6,Dixie County,FL
1779,"Holmes County, FL",0.234000,6.6,0.9,0.7,0.1,2.0,2.7,87.2,Holmes County,FL
1782,"Nassau County, FL",0.233596,6.5,0.5,1.0,0.1,1.5,3.8,87.2,Nassau County,FL
1791,"Gilchrist County, FL",0.231825,5.6,0.6,0.5,0.1,1.2,5.2,87.3,Gilchrist County,FL


In [36]:
#test to make sure leading space is gone
df2[df2['State'] == 'FL']

,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,County,State
31,"Broward County, FL",0.678237,28.5,0.4,3.6,0.1,2.0,26.9,40.8,Broward County,FL
34,"Orange County, FL",0.671218,22.0,0.6,5.4,0.2,2.4,28.7,44.1,Orange County,FL
71,"Hillsborough County, FL",0.626423,17.4,0.5,3.9,0.1,2.5,26.0,52.3,Hillsborough County,FL
96,"Hendry County, FL",0.609280,13.4,2.2,1.1,0.2,1.1,50.3,34.5,Hendry County,FL
103,"Osceola County, FL",0.605921,13.1,0.8,3.0,0.2,2.5,48.6,37.3,Osceola County,FL
...,...,...,...,...,...,...,...,...,...,...,...
1663,"Dixie County, FL",0.257861,8.9,0.4,0.4,0.0,1.5,3.5,85.6,Dixie County,FL
1779,"Holmes County, FL",0.234000,6.6,0.9,0.7,0.1,2.0,2.7,87.2,Holmes County,FL
1782,"Nassau County, FL",0.233596,6.5,0.5,1.0,0.1,1.5,3.8,87.2,Nassau County,FL
1791,"Gilchrist County, FL",0.231825,5.6,0.6,0.5,0.1,1.2,5.2,87.3,Gilchrist County,FL


In [37]:
#remove the word 'County' from county column, so we can merge tables.
df2['County'] = df2['County'].str.replace(r'County', '')


In [38]:
df2

,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,County,State
31,"Broward County, FL",0.678237,28.5,0.4,3.6,0.1,2.0,26.9,40.8,Broward,FL
34,"Orange County, FL",0.671218,22.0,0.6,5.4,0.2,2.4,28.7,44.1,Orange,FL
71,"Hillsborough County, FL",0.626423,17.4,0.5,3.9,0.1,2.5,26.0,52.3,Hillsborough,FL
96,"Hendry County, FL",0.609280,13.4,2.2,1.1,0.2,1.1,50.3,34.5,Hendry,FL
103,"Osceola County, FL",0.605921,13.1,0.8,3.0,0.2,2.5,48.6,37.3,Osceola,FL
...,...,...,...,...,...,...,...,...,...,...,...
1663,"Dixie County, FL",0.257861,8.9,0.4,0.4,0.0,1.5,3.5,85.6,Dixie,FL
1779,"Holmes County, FL",0.234000,6.6,0.9,0.7,0.1,2.0,2.7,87.2,Holmes,FL
1782,"Nassau County, FL",0.233596,6.5,0.5,1.0,0.1,1.5,3.8,87.2,Nassau,FL
1791,"Gilchrist County, FL",0.231825,5.6,0.6,0.5,0.1,1.2,5.2,87.3,Gilchrist,FL


In [40]:
#remove extra space following county name
df2['County'] = df2['County'].str.rstrip()

In [42]:

df2[df2['County'] == 'Broward']

,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,County,State
31,"Broward County, FL",0.678237,28.5,0.4,3.6,0.1,2.0,26.9,40.8,Broward,FL


# MERGE THE DATASETS

In [81]:
#perform the merge

In [45]:
merged = pd.merge(df1,df2, on='County', how='outer')

In [46]:
merged

,County,Province_State,Country_Region,Date,Confirmed,Deaths,Location,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,State
0,Alachua,Florida,US,4/9/20,218,0,"Alachua County, FL",0.547155,20.4,0.3,5.7,0.1,2.5,8.9,63.2,FL
1,Baker,Florida,US,4/9/20,17,3,"Baker County, FL",0.307153,13.4,0.4,0.7,0.0,1.6,2.3,82.1,FL
2,Bay,Florida,US,4/9/20,57,2,"Bay County, FL",0.371522,11.2,0.8,2.3,0.1,3.0,5.4,78.2,FL
3,Bradford,Florida,US,4/9/20,42,2,"Bradford County, FL",0.398624,19.3,0.4,0.5,0.1,1.7,3.6,75.0,FL
4,Brevard,Florida,US,4/9/20,218,7,"Brevard County, FL",0.395788,10.6,0.5,2.3,0.1,2.5,9.0,76.4,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Union,Florida,US,4/9/20,3,0,"Union County, FL",0.446389,22.5,0.4,0.3,0.0,1.4,5.4,70.7,FL
65,Volusia,Florida,US,4/9/20,342,10,"Volusia County, FL",0.420775,11.0,0.5,1.7,0.1,1.9,12.0,74.3,FL
66,Wakulla,Florida,US,4/9/20,15,2,"Wakulla County, FL",0.354523,15.1,0.7,0.5,0.1,1.9,3.6,78.8,FL
67,Walton,Florida,US,4/9/20,32,0,"Walton County, FL",0.284614,5.9,1.0,1.0,0.2,2.4,6.2,84.1,FL


In [84]:
# rename a column and drop 'location columns'

In [47]:
merged = merged.rename(columns = {'Confirmed' : 'Confirmed Cases'})

In [48]:
merged = merged.drop('Location', axis=1)

In [50]:
#end result
merged

,County,Province_State,Country_Region,Date,Confirmed Cases,Deaths,Diversity-Index,Black,Native American,Asian,Pacific Islander,Plus two races,Hispanic,White,State
0,Alachua,Florida,US,4/9/20,218,0,0.547155,20.4,0.3,5.7,0.1,2.5,8.9,63.2,FL
1,Baker,Florida,US,4/9/20,17,3,0.307153,13.4,0.4,0.7,0.0,1.6,2.3,82.1,FL
2,Bay,Florida,US,4/9/20,57,2,0.371522,11.2,0.8,2.3,0.1,3.0,5.4,78.2,FL
3,Bradford,Florida,US,4/9/20,42,2,0.398624,19.3,0.4,0.5,0.1,1.7,3.6,75.0,FL
4,Brevard,Florida,US,4/9/20,218,7,0.395788,10.6,0.5,2.3,0.1,2.5,9.0,76.4,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Union,Florida,US,4/9/20,3,0,0.446389,22.5,0.4,0.3,0.0,1.4,5.4,70.7,FL
65,Volusia,Florida,US,4/9/20,342,10,0.420775,11.0,0.5,1.7,0.1,1.9,12.0,74.3,FL
66,Wakulla,Florida,US,4/9/20,15,2,0.354523,15.1,0.7,0.5,0.1,1.9,3.6,78.8,FL
67,Walton,Florida,US,4/9/20,32,0,0.284614,5.9,1.0,1.0,0.2,2.4,6.2,84.1,FL


In [52]:
#final rename for loading to database
merged = merged.rename(columns={'County':'county',\
                                'Confirmed Cases' : 'confirmedcases',\
                                'Deaths' : 'deaths',\
                                'Diversity-Index' : 'diversityindex',\
                                'Black' : 'black',\
                                'Native American' : 'nativeamerican',\
                                'Asian' : 'asian',\
                                'Pacific Islander' : 'pacificislander',\
                                'Plus two races' : 'plustworaces',\
                                'Hispanic': 'hispanic',\
                                'White' : 'white',\
                                'State': 'state'})                          
                                
                                
    


In [53]:
merged.head()

,county,Province_State,Country_Region,Date,confirmedcases,deaths,diversityindex,black,nativeamerican,asian,pacificislander,plustworaces,hispanic,white,state
0,Alachua,Florida,US,4/9/20,218,0,0.547155,20.4,0.3,5.7,0.1,2.5,8.9,63.2,FL
1,Baker,Florida,US,4/9/20,17,3,0.307153,13.4,0.4,0.7,0.0,1.6,2.3,82.1,FL
2,Bay,Florida,US,4/9/20,57,2,0.371522,11.2,0.8,2.3,0.1,3.0,5.4,78.2,FL
3,Bradford,Florida,US,4/9/20,42,2,0.398624,19.3,0.4,0.5,0.1,1.7,3.6,75.0,FL
4,Brevard,Florida,US,4/9/20,218,7,0.395788,10.6,0.5,2.3,0.1,2.5,9.0,76.4,FL


In [55]:
#drop the final unnecessary columns
merged = merged.drop(['Province_State', 'Country_Region','Date'], axis=1)

In [56]:
merged

,county,confirmedcases,deaths,diversityindex,black,nativeamerican,asian,pacificislander,plustworaces,hispanic,white,state
0,Alachua,218,0,0.547155,20.4,0.3,5.7,0.1,2.5,8.9,63.2,FL
1,Baker,17,3,0.307153,13.4,0.4,0.7,0.0,1.6,2.3,82.1,FL
2,Bay,57,2,0.371522,11.2,0.8,2.3,0.1,3.0,5.4,78.2,FL
3,Bradford,42,2,0.398624,19.3,0.4,0.5,0.1,1.7,3.6,75.0,FL
4,Brevard,218,7,0.395788,10.6,0.5,2.3,0.1,2.5,9.0,76.4,FL
...,...,...,...,...,...,...,...,...,...,...,...,...
64,Union,3,0,0.446389,22.5,0.4,0.3,0.0,1.4,5.4,70.7,FL
65,Volusia,342,10,0.420775,11.0,0.5,1.7,0.1,1.9,12.0,74.3,FL
66,Wakulla,15,2,0.354523,15.1,0.7,0.5,0.1,1.9,3.6,78.8,FL
67,Walton,32,0,0.284614,5.9,1.0,1.0,0.2,2.4,6.2,84.1,FL


### Create database connection

In [57]:
connection_string = "postgres:Password123@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{connection_string}')

In [58]:
# Confirm tables
engine.table_names()

['countycovid']

### Load DataFrames into database

In [60]:
merged.to_sql(name='countycovid', con=engine, if_exists='append', index=False)